In [33]:
import pandas as pd
import os
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import sklearn.metrics
from collections import defaultdict
import shutil

pd.reset_option('all')

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



In [34]:
table = '/Users/rijulvohra/Documents/work/Novartis-ISI/table_linker_pipeline/'
table_name = 'players.csv'
wikify_column_name = "Player (2011 TBs)"

In [35]:
#intermediate files
canonical = 'canonical.csv'
candidates = "candidates.csv"
feature_votes = "feature_votes.csv"
score_file = "scores.csv"
temp_dir = os.path.join(os.path.join(table,'temp'))
table_path = os.path.join(table,table_name)
if not(os.path.exists(temp_dir)):
    os.makedirs(temp_dir)

## Canonicalize

In [36]:
canonical_file = os.path.join(temp_dir,canonical)
!tl canonicalize -c "$wikify_column_name" --add-context $table_path > $canonical_file

In [37]:
pd.read_csv(canonical_file, nrows = 10)

,column,row,label,context
0,1,0,Hank Aaron,"1|6,856"
1,1,1,Stan Musial,"2|6,134"
2,1,2,Willie Mays,"3|6,066"
3,1,3,Barry Bonds,"4|5,976"
4,1,4,Ty Cobb,"5|5,854"
5,1,5,Babe Ruth,"6|5,793"
6,1,6,Pete Rose,"7|5,752"
7,1,7,Carl Yastrzemski,"8|5,539"
8,1,8,Eddie Murray,"9|5,397"
9,1,9,Rafael Palmeiro,"10|5,388"


## Candidate Generation

In [38]:
candidates_file = os.path.join(temp_dir,candidates)
!tl clean -c label -o label_clean $canonical_file / --url http://ckg06:9200 --index wikidata_augmented_index \
                                                    get-fuzzy-augmented-matches -c label_clean \
        / --url http://ckg06:9200 --index wikidataos-07 get-exact-matches -c label_clean > $candidates_file

In [39]:
pd.read_csv(candidates_file,nrows = 10)

,column,row,label,context,label_clean,kg_id,kg_labels,method,kg_descriptions,pagerank,retrieval_score
0,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,painting by Ross R. Rossin,6.993175e-09,30.307909
1,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q215777,Hank Aaron|Henry Louis Aaron|Henry Hank L. Aaron,fuzzy-augmented,American baseball player,1.859623e-08,30.307909
2,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,NaN,4.207063e-08,28.132477
3,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,annual baseball award given to the top hitter ...,8.273086e-09,28.132477
4,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q92433937,Reflections on Hank Aaron,fuzzy-augmented,scientific article published on 29 April 2019,0.000000e+00,26.248425
5,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,NaN,0.000000e+00,26.248425
6,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q5648263,Hank Aaron: Chasing the Dream,fuzzy-augmented,1995 film by Michael Tollin,6.993175e-09,24.600891
7,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q66847614,President Carter with Hank Aaron (NAID 180805),fuzzy-augmented,file unit in the National Archives and Records...,6.993175e-09,21.857079
8,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q16983107,Oak Leaf Trail,fuzzy-augmented,NaN,6.993175e-09,19.045750
9,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q32826649,"Rich Hank, Poor Hank",fuzzy-augmented,episode of TV series King of the Hill,9.236274e-09,18.247020


## Feature Voting

In [40]:
%%time
feature_votes_file = os.path.join(temp_dir,feature_votes)
!tl smallest-qnode-number $candidates_file \
    / string-similarity -i --method monge_elkan:tokenizer=word -o monge_elkan \
    / string-similarity -i --method jaccard:tokenizer=word -c kg_descriptions context -o des_cont_jaccard \
    / feature-voting -c "pagerank,smallest_qnode_number,monge_elkan,des_cont_jaccard" > $feature_votes_file

CPU times: user 71 ms, sys: 31.2 ms, total: 102 ms
Wall time: 4.78 s


In [41]:
pd.read_csv(feature_votes_file,nrows = 10)

,column,row,label,context,label_clean,kg_id,kg_labels,method,kg_descriptions,pagerank,retrieval_score,smallest_qnode_number,monge_elkan,des_cont_jaccard,votes
0,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,painting by Ross R. Rossin,6.993175e-09,30.307909,0,1.000000,0,1
1,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q215777,Hank Aaron|Henry Louis Aaron|Henry Hank L. Aaron,fuzzy-augmented,American baseball player,1.859623e-08,30.307909,0,1.000000,0,1
2,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,NaN,4.207063e-08,28.132477,0,1.000000,0,1
3,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,annual baseball award given to the top hitter ...,8.273086e-09,28.132477,0,1.000000,0,1
4,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q92433937,Reflections on Hank Aaron,fuzzy-augmented,scientific article published on 29 April 2019,0.000000e+00,26.248425,0,1.000000,0,1
5,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,NaN,0.000000e+00,26.248425,0,1.000000,0,1
6,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q5648263,Hank Aaron: Chasing the Dream,fuzzy-augmented,1995 film by Michael Tollin,6.993175e-09,24.600891,0,0.983333,0,0
7,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q66847614,President Carter with Hank Aaron (NAID 180805),fuzzy-augmented,file unit in the National Archives and Records...,6.993175e-09,21.857079,0,1.000000,0,1
8,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q16983107,Oak Leaf Trail,fuzzy-augmented,NaN,6.993175e-09,19.045750,0,0.616667,0,0
9,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q32826649,"Rich Hank, Poor Hank",fuzzy-augmented,episode of TV series King of the Hill,9.236274e-09,18.247020,0,0.816667,0,0


## Compute Embedding Score using Column Vector Strategy

In [42]:
%%time
score = os.path.join(temp_dir,score_file)
!tl score-using-embedding $feature_votes_file --column-vector-strategy centroid-of-singletons -o graph-embedding-score \
--embedding-file /tmp/wikidataos-graph-embedding-01.tsv \
--embedding-url http://kg2018a.isi.edu:9200/wikidataos-graph-embedding-01/doc/ \
> $score

Qnodes to lookup: 3035
Qnodes from file: 2896
Qnodes from server: 0
CPU times: user 30.9 ms, sys: 19.1 ms, total: 50 ms
Wall time: 2 s


In [43]:
pd.read_csv(score,nrows = 10)

,column,row,label,context,label_clean,kg_id,kg_labels,method,kg_descriptions,pagerank,retrieval_score,smallest_qnode_number,monge_elkan,des_cont_jaccard,votes,graph-embedding-score
0,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q47513596,Hank Aaron,fuzzy-augmented,painting by Ross R. Rossin,6.993175e-09,30.307909,0,1.000000,0,1,0.356457
1,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q215777,Hank Aaron|Henry Louis Aaron|Henry Hank L. Aaron,fuzzy-augmented,American baseball player,1.859623e-08,30.307909,0,1.000000,0,1,0.839375
2,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q14679126,Hank Aaron Stadium,fuzzy-augmented,NaN,4.207063e-08,28.132477,0,1.000000,0,1,0.432508
3,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q1518478,Hank Aaron Award,fuzzy-augmented,annual baseball award given to the top hitter ...,8.273086e-09,28.132477,0,1.000000,0,1,0.552475
4,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q92433937,Reflections on Hank Aaron,fuzzy-augmented,scientific article published on 29 April 2019,0.000000e+00,26.248425,0,1.000000,0,1,NaN
5,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q28453830,Hank Aaron State Trail,fuzzy-augmented,NaN,0.000000e+00,26.248425,0,1.000000,0,1,NaN
6,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q5648263,Hank Aaron: Chasing the Dream,fuzzy-augmented,1995 film by Michael Tollin,6.993175e-09,24.600891,0,0.983333,0,0,0.324928
7,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q66847614,President Carter with Hank Aaron (NAID 180805),fuzzy-augmented,file unit in the National Archives and Records...,6.993175e-09,21.857079,0,1.000000,0,1,0.266241
8,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q16983107,Oak Leaf Trail,fuzzy-augmented,NaN,6.993175e-09,19.045750,0,0.616667,0,0,0.382838
9,1,0,Hank Aaron,"1|6,856",Hank Aaron,Q32826649,"Rich Hank, Poor Hank",fuzzy-augmented,episode of TV series King of the Hill,9.236274e-09,18.247020,0,0.816667,0,0,0.342757


## Compute final score

In [44]:
final_score = os.path.join(table,'final_score.csv')
!tl normalize-scores -c graph-embedding-score -t zscore -o normalized-graph-embedding-score $score \
    / normalize-scores -c pagerank -t zscore -o normalized-pagerank \
    / normalize-scores -c monge_elkan -t zscore -o normalized-monge-elkan > $final_score
df = pd.read_csv(final_score)

In [45]:
def calc_sum(x):
    graph_score = x['normalized-graph-embedding-score']
    pagerank = x['normalized-pagerank']
    monge_elkan = x['normalized-monge-elkan']
    l = [graph_score,pagerank,monge_elkan]
    l1 = list(filter(lambda x: not(pd.isna(x)),l))
    ans = sum(l1)
    return ans
df['final_score'] = df.apply(lambda x: calc_sum(x),axis = 1)

## Get Top5 KG Links

In [46]:
top5_df = df[df['method'] == 'fuzzy-augmented'].groupby(['column','row']).apply(lambda x: x.sort_values(['final_score'],ascending=False)[:5]).reset_index(drop = True)

In [47]:
def get_kg_links(top5_df,input_df):
    k = defaultdict(list)
    for i, row in top5_df.iterrows():
        col_row_key = str(row['column']) + '_' + str(row['row'])
        k[col_row_key].append((row['kg_id'],row['kg_labels'],row['graph-embedding-score'],
                               row['monge_elkan'],row['pagerank'],row['final_score']))
        
    output_df_list = []
    for key in k:
        col,row = int(key.split('_')[0]),int(key.split('_')[1])
        wikify_col = input_df.columns[col]
        kg_id = wikify_col + '_kg_id'
        kg_label = wikify_col + '_kg_label'
        kg_score = wikify_col + '_score'
        for val in k[key]:
            _ = {}
            for c in input_df.columns:
                _[c] = input_df.loc[row,c]
            _[kg_id] = val[0]
            _[kg_label] = val[1]
            _['graph-embedding-score'] = val[2]
            _['monge_elkan'] = val[3]
            _['pagerank'] = val[4]
            _[kg_score] = val[5]
            output_df_list.append(_)
    output_df = pd.DataFrame(output_df_list)
    
    return output_df

input_df = pd.read_csv(table_path)
output_df = get_kg_links(top5_df, input_df)

In [48]:
output_df.head(20)

,Rank,Player (2011 TBs),TBs,Player (2011 TBs)_kg_id,Player (2011 TBs)_kg_label,graph-embedding-score,monge_elkan,pagerank,Player (2011 TBs)_score
0,1,Hank Aaron,"6,856",Q215777,Hank Aaron|Henry Louis Aaron|Henry Hank L. Aaron,0.839375,1.000000,1.859623e-08,3.721903
1,1,Hank Aaron,"6,856",Q5648399,Hank Mason,0.826037,0.866667,6.993175e-09,2.927188
2,1,Hank Aaron,"6,856",Q2662383,Hank Thompson,0.855141,0.816667,6.993175e-09,2.863069
3,1,Hank Aaron,"6,856",Q866176,Hank Conger|Hyun Choi Conger,0.852715,0.816667,6.993175e-09,2.847020
4,1,Hank Aaron,"6,856",Q687282,Hank Greenberg|Henry Benjamin Hank Greenberg,0.818661,0.816667,8.042763e-09,2.623815
5,2,Stan Musial,"6,134",Q1328142,Stan Musial|Stash|The Donora Greyhound|Stanley...,0.759413,1.000000,1.115597e-08,3.178807
6,2,Stan Musial,"6,134",Q7599195,"Stanisław Musiał|Stanisław Musiał, sj",0.591376,0.911111,7.401555e-09,1.604120
7,2,Stan Musial,"6,134",Q16025125,Joe Musial,0.634521,0.805556,6.993175e-09,1.346968
8,2,Stan Musial,"6,134",Q16194347,Marius Stan,0.551735,0.890000,7.272838e-09,1.233362
9,2,Stan Musial,"6,134",Q47502630,Mircea Stan,0.593354,0.833333,6.993175e-09,1.217263


In [49]:
linked_table_name = "linked-" + table_name
output_path = os.path.join(table,linked_table_name)
output_df.to_csv(output_path,index = False)

## CleanUp temporary files

In [66]:
shutil.rmtree(temp_dir)